# Agents & Docling MCP with Llama Stack

This notebook highlights some use cases for agents that want to leverage **Docling MCP** tools to complete tasks in document understanding.

We will use the Llama Stack framework. To get an introduction to Llama Stack capabilities, including its current builtin tools, you can refer to the
[Llama Stack Demos on OpenDataHub](https://github.com/opendatahub-io/llama-stack-demos) repository.

### Agent Example:

This notebook is a simple example to build a system that can convert PDF files (or any other document format supported by Docling) and ran basic transformation operations on the converted format `DoclingDocument`

### Docling MCP Tools:

We will use the tools from the [Docling MCP] server that allow executing tasks such as:
- converting a PDF file from a local or remote location into a unified document representation [DoclingDocument](https://docling-project.github.io/docling/concepts/docling_document/).
- exporting the `DoclingDocument` to a text format (markdown).
- adding new content to a `DoclingDocument`.
- saving a `DoclingDocument` to a file in markdown format.

## Pre-Requisites

Before starting this notebook, ensure that you have:
- Followed the instructions in the [README.md](./README.md) file to set up the following resources:
  - Inference model with Ollama
  - Llama Stack server with the Ollama template [distribution-llama](https://hub.docker.com/r/llamastack/distribution-ollama)
  - Docling MCP server 

You may want to create a virtual environment to run this notebook. The [requirements.txt](./requirements.txt) file contains the minimal requirements.

```bash
python -m venv venv
source venv/bin/activate
pip install --upgrade pip -r requirements.txt
```


## Setting Up this Notebook

Rename or copy the [`.env.example`](./.env.examples) file to create a new file called `.env`. Most environmental variables are already set up with default values to run this notebook and they are aligned to the set up of the pre-requisites, like the Llama Stack server and the Docling MCP endpoints.

```bash
cp .env.example .env
```

### Environment variables required for this notebook

- `BASE_URL`: the URL of the remote Llama Stack server. Defaults to `http://localhost:8321`.
- `INFERENCE_MODEL`: the generative AI model id. Defaults to the Meta Llama 3.2 model (`meta-llama/Llama-3.2-3B-Instruct`).
- `TEMPERATURE` (optional): the temperature to use during inference. Defaults to 0.0.
- `TOP_P` (optional): the top_p parameter to use during inference. Defaults to 0.95.
- `MAX_TOKENS` (optional): the maximum number of tokens that can be generated in the completion. Defaults to 4096.
- `STREAM` (optional): set this to True to stream the output of the model/agent and False otherwise. Defaults to True.
- `USE_PROMPT_CHAINING`: dictates if the prompt should be formatted as a few separate prompts to isolate each step or in a single turn.
- `DOCLING_MCP_URL`: the URL for the Docling MCP server. If the client does not find the tool registered to the llama-stack instance, it will use this URL to register it.

## Necessary Imports

In [1]:
import logging
import uuid

from llama_stack_client import Agent
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from pydantic import NonNegativeFloat, AnyHttpUrl
from pydantic_settings import BaseSettings, SettingsConfigDict

from src.utils import step_printer, user_printer

# set the logger
logger = logging.getLogger(__name__)
if not logger.hasHandlers():
    logger.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter("%(message)s")
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)


# access the environment variables
class Settings(BaseSettings):
    base_url: AnyHttpUrl
    inference_model: str
    max_tokens: int
    temperature: NonNegativeFloat
    top_p: float
    stream: bool
    use_prompt_chaining: bool
    docling_mcp_url: AnyHttpUrl

    model_config = SettingsConfigDict(env_file=".env", env_file_encoding="utf-8")


settings = Settings(
    base_url="http://localhost:8321",
    inference_model="meta-llama/Llama-3.2-3B-Instruct",
    temperature=0.0,
    max_tokens=4096,
    top_p=0.95,
    stream=False,
    use_prompt_chaining=True,
    docling_mcp_url="http://host.containers.internal:8000/sse",
)
print(settings)

base_url=AnyHttpUrl('http://localhost:8321/') inference_model='meta-llama/Llama-3.2-3B-Instruct' max_tokens=4096 temperature=0.0 top_p=0.95 stream=False use_prompt_chaining=True docling_mcp_url=AnyHttpUrl('http://host.containers.internal:8000/sse')


## Setting Up the Server Connection

Establish the connection to your Llama Stack server.

In [2]:
client = LlamaStackClient(base_url=str(settings.base_url))
print(f"Connected to Llama Stack server @ {client.base_url}")

Connected to Llama Stack server @ http://localhost:8321/


## Initializing the Inference Parameters

Fetch the inference-related parameters from the corresponding environment variables and convert them to the format Llama Stack expects.

In [3]:
if settings.temperature > 0.0:
    strategy = {
        "type": "top_p",
        "temperature": settings.temperature,
        "top_p": settings.top_p,
    }
else:
    strategy = {"type": "greedy"}

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": settings.max_tokens,
}

print(
    f"Inference Parameters:\n\tSampling Parameters: {sampling_params}\n\tstream: {settings.stream}"
)

Inference Parameters:
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: False


## Validate that the Docling MCP tools are available in the Llama Stack instance

When an instance of Llama Stack is redeployed, it may be the case that the tools will need to be re-registered. Also if a tool is already registered with a Llama Stack instance, trying to register another one with the same `toolgroup_id` will throw you an error.

For this reason, it is recommended to validate your tools and toolgroups. The following code will check that `mcp::docling` tools are correctly registered, and if not it will attempt to register them using their specific endpoints.

In [5]:
registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
if "mcp::docling" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::docling",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri": str(settings.docling_mcp_url)},
    )

registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
print(
    f"Your Llama Stack server is registered with the following tool groups @ {set(registered_toolgroups)} \n"
)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 200 OK"


Your Llama Stack server is registered with the following tool groups @ {'builtin::wolfram_alpha', 'builtin::rag', 'mcp::docling', 'builtin::websearch'} 



## Defining our Agent - Prompt Chaining

We define an agent provided with the **Docling MCP** tools. The agent should be able to accomplish the following tasks in a multi-step, multi-tool approach:

1. Converting a PDF file into the `DoclingDocument` format.
2. Exporting the result in a text format, such as markdown.
3. Generating a summary of the document content.

In [6]:
model_prompt = """You are a helpful assistant. You have access to a number of tools.
Whenever a tool is called, be sure to return the Response in a friendly and helpful tone.
"""

In [7]:
# Create simple agent with tools
agent = Agent(
    client=client,
    model=settings.inference_model,  # replace this with model_id to get the value of INFERENCE_MODEL_ID environment variable
    instructions=model_prompt,  # update system prompt based on the model you are using
    tools=["mcp::docling"],
    tool_config={"tool_choice": "auto"},
    sampling_params=sampling_params,
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 200 OK"


In [8]:
user_prompts = [
    "Convert the PDF document on https://arxiv.org/pdf/2408.09869 to DoclingDocument.",
    "Export the document to markdown.",
    "Write a summary of the document content.",
]
session_id = agent.create_session(f"docling-session_{uuid.uuid4()}")

for i, prompt in enumerate(user_prompts):
    user_printer(prompt)
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=settings.stream,
    )
    if settings.stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(
            response.steps
        )  # print the steps of an agent's response in a formatted way.

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/bf6de0a0-3fcc-46c6-bc9b-3a33ce32f1e2/session "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/bf6de0a0-3fcc-46c6-bc9b-3a33ce32f1e2/session/a9546fef-6613-4423-a4a2-e10db0bd1e4b/turn "HTTP/1.1 200 OK"


👤 User Query:
Convert the PDF document on https://arxiv.org/pdf/2408.09869 to DoclingDocument.

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: convert_pdf_document_into_docling_document, Arguments: {'source': 'https://arxiv.org/pdf/2408.09869'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='{\n  "success": true,\n  "document_key": "2679dacbd19787abbd565a7c6b46a37a"\n}',
│   │   type='text'
│   )
]

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/bf6de0a0-3fcc-46c6-bc9b-3a33ce32f1e2/session/a9546fef-6613-4423-a4a2-e10db0bd1e4b/turn "HTTP/1.1 200 OK"



---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The conversion of the PDF document from https://arxiv.org/pdf/2408.09869 to DoclingDocument was successful. The resulting document has a unique key '2679dacbd19787abbd565a7c6b46a37a'. You can use this key to access and manipulate the converted document in the local cache.

You can now use the `export_docling_document_to_markdown` function with the document key to view the converted document in markdown format.

========== Query processing completed ========== 

👤 User Query:
Export the document to markdown.

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: export_docling_document_to_markdown, Arguments: {'document_key': '2679dacbd19787abbd565a7c6b46a37a'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='{\n  "document_key": "2679dacbd19787abbd565a7c6b46a37a",\n  "markdown": "<!-- image -->\\n\\n## Docling Technical Report\\n\\nVersion 1.0\\n\\nChristoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar\\n\\nAI4K Group, IBM Research R¨ uschlikon, Switzerland\\n\\n## Abstract\\n\\nThis technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. The code interface allows for easy extensibility and addition of new features and models.\\n\\n## 1 Introduction\\n\\nConverting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge variability in formats, weak standardization and printing-optimized characteristic, which discards most structural features and metadata. With the advent of LLMs and popular application patterns such as retrieval-augmented generation (RAG), leveraging the rich content embedded in PDFs has become ever more relevant. In the past decade, several powerful document understanding solutions have emerged on the market, most of which are commercial software, cloud offerings [3] and most recently, multi-modal vision-language models. As of today, only a handful of open-source tools cover PDF conversion, leaving a significant feature and quality gap to proprietary solutions.\\n\\nWith Docling , we open-source a very capable and efficient document conversion tool which builds on the powerful, specialized AI models and datasets for layout analysis and table structure recognition we developed and presented in the recent past [12, 13, 9]. Docling is designed as a simple, self-contained python library with permissive license, running entirely locally on commodity hardware. Its code architecture allows for easy extensibility and addition of new features and models.\\n\\nHere is what Docling delivers today:\\n\\n- Converts PDF documents to JSON or Markdown format, stable and lightning fast\\n- Understands detailed page layout, reading order, locates figures and recovers table structures\\n- Extracts metadata from the document, such as title, authors, references and language\\n- Optionally applies OCR, e.g. for scanned PDFs\\n- Can be configured to be optimal for batch-mode (i.e high throughput, low time-to-solution) or interactive mode (compromise on efficiency, low time-to-solution)\\n- Can leverage different accelerators (GPU, MPS, etc).\\n\\n## 2 Getting Started\\n\\nTo use Docling, you can simply install the docling package from PyPI. Documentation and examples are available in our GitHub repository at github.com/DS4SD/docling. All required model assets 1 are downloaded to a local huggingface datasets cache on first use, unless you choose to pre-install the model assets in advance.\\n\\nDocling provides an easy code interface to convert PDF documents from file system, URLs or binary streams, and retrieve the output in either JSON or Markdown format. For convenience, separate methods are offered to convert single documents or batches of documents. A basic usage example is illustrated below. Further examples are available in the Doclign code repository.\\n\\n```\\nfrom docling.document_converter import DocumentConverter Large\\n```\\n\\n```\\nsource = \\"https://arxiv.org/pdf/2206.01062\\" # PDF path or URL converter = DocumentConverter() result = converter.convert_single(source) print(result.render_as_markdown()) # output: \\"## DocLayNet: A Human -Annotated Dataset for Document -Layout Analysis [...]\\"\\n```\\n\\nOptionally, yo

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/bf6de0a0-3fcc-46c6-bc9b-3a33ce32f1e2/session/a9546fef-6613-4423-a4a2-e10db0bd1e4b/turn "HTTP/1.1 200 OK"



---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
This is a research paper on a dataset called DocLayNet, which is a collection of annotated documents for natural language processing tasks. The paper describes the creation and characteristics of the dataset, including its size, annotation process, and quality control measures.

Here are some key points from the paper:

1. **Dataset size**: The DocLayNet dataset consists of 1,107,470 documents, with a total of 10,714,190 annotated instances.
2. **Annotation process**: The annotations were performed by a team of 40 dedicated annotators, who worked in three phases: data selection and preparation, annotation, and quality control.
3. **Labeling scheme**: The dataset includes 11 labels, including:
	* Caption
	* Footnote
	* Formula
	* List item
	* Page header
	* Picture
	* Section header
	* Table
	* Text
	* Title
	* Total
4. **Annotation guidelines**: Annotators were instructed to use a specific labeling scheme, which ensures t

## Defining our Agent - ReAct

We can also define our agent to be more autonomous and perform the same task with a single prompt instead of a chain. To do this, we leverage Llama Stack's **ReAct agent**, which has the ability to loop through the *Reason then Act* iterations, thinking through the problem and then using tools until it determines that it's task has been completed successfully.  

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In this example, we will leverage the generation tools of **Docling MCP** to:
- create a new `DoclingDocument`
- add a title and paragraphs
- export the resulting document into a file in markdown format

The purpose of the code below is just to show the instructions sent to the LLM to complete the task. They incorporate the description of the Docling MCP tools.

In [9]:
# ReAct instructions
from llama_stack_client.lib.agents.agent import AgentUtils
from llama_stack_client.lib.agents.react.agent import get_default_react_instructions

client_tools = AgentUtils.get_client_tools(["mcp::docling"])
instructions = get_default_react_instructions(client, ["mcp::docling"], client_tools)
print(instructions)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 200 OK"



You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to the following tools: is_document_in_local_cache, convert_pdf_document_into_docling_document, create_new_docling_document, export_docling_document_to_markdown, save_docling_document, add_title_to_docling_document, add_section_heading_to_docling_document, add_paragraph_to_docling_document, open_list_in_docling_document, close_list_in_docling_document, add_list_items_to_list_in_docling_document, add_table_in_html_format_to_docling_document, get_overview_of_document_anchors, get_text_of_document_item_at_anchor, update_text_of_document_item_at_anchor, delete_document_items_at_anchors

You must always respond in the following JSON format:
{
    "thought": $THOUGHT_PROCESS,
    "action": {
        "tool_name": $TOOL_NAME,
        "tool_params": $TOOL_PARAMS
    },
    "answer": $ANSWER
}

Specifically, this json should have a `thought

In [32]:
agent = ReActAgent(
    client=client,
    model=settings.inference_model,
    tools=["mcp::docling"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=sampling_params,
)

user_prompts = [
    """I want you to create a new Docling document on Quantum Computing.
    To do this, you will create a document first by invoking `create_new_docling_document` with Quantum Computing as prompt.
    Add a title and 3 paragraphs describing the topic, one by one.
    At the end of the writing, you must save the document by calling `save_docling_document` and return me the filepath of
    the saved document in markdown format.
    """
]

session_id = agent.create_session(f"docling-session_{uuid.uuid4()}")
for prompt in user_prompts:
    print("\n" + "=" * 50)
    user_printer(prompt)
    print("=" * 50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=settings.stream,
    )
    if settings.stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(
            response.steps
        )  # print the steps of an agent's response in a formatted way.

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/06eb6a2d-ca0b-473f-be27-75240d1e659a/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/06eb6a2d-ca0b-473f-be27-75240d1e659a/session/b9233abc-1f44-4850-a308-be5381c4f0d9/turn "HTTP/1.1 200 OK"



👤 User Query:
I want you to create a new Docling document on Quantum Computing.
    To do this, you will create a document first by invoking `create_new_docling_document` with Quantum Computing as prompt.
    Add a title and 3 paragraphs describing the topic, one by one.
    At the end of the writing, you must save the document by calling `save_docling_document` and return me the filepath of
    the saved document in markdown format.
    


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/invoke "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/06eb6a2d-ca0b-473f-be27-75240d1e659a/session/b9233abc-1f44-4850-a308-be5381c4f0d9/turn/606cdfb8-3c94-4822-a016-c134de43b2b2/resume "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/invoke "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/06eb6a2d-ca0b-473f-be27-75240d1e659a/session/b9233abc-1f44-4850-a308-be5381c4f0d9/turn/606cdfb8-3c94-4822-a016-c134de43b2b2/resume "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/invoke "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/06eb6a2d-ca0b-473f-be27-75240d1e659a/session/b9233abc-1f44-4850-a308-be5381c4f0d9/turn/606cdfb8-3c94-4822-a016-c134de43b2b2/resume "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/invoke "HTTP/1.1 200


---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
  "thought": "I want to create a new Docling document on Quantum Computing",
  "action": {
    "tool_name": "create_new_docling_document",
    "tool_params": [
      {"name": "prompt", "value": "Quantum Computing"}
    ]
  },
  "answer": null
}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


...


---------- 📍 Step 9: InferenceStep ----------
🤖 Model Response:
{
  "thought": "I want to add a title and three paragraphs to the document",
  "action": {
    "tool_name": "add_paragraph_to_docling_document",
    "tool_params": [
      {"name": "document_key", "value": "6f1933d17fb2cce1d2a578e40315f711"},
      {"name": "paragraph", "value": "Quantum Computing: An Introduction"}
    ]
  },
  "answer": null
}


---------- 📍 Step 10: ToolExecutionStep ----------
🔧 Executing tool...


[TextContentItem(text='{\n  "document_key": "6f1933d17fb2cce1d2a578e40315f711"\n}', type='text')]


---------- 📍 Step 11: InferenceStep ----------
🤖 Model Response:
{
  "thought": "I want to save the document",
  "action": {
    "tool_name": "save_docling_document",
    "tool_params": [
      {"name": "document_key", "value": "6f1933d17fb2cce1d2a578e40315f711"}
    ]
  },
  "answer": null
}


---------- 📍 Step 12: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='{\n  "md_file": "/Users/ceb/git/docling-mcp/_cache/6f1933d17fb2cce1d2a578e40315f711.md",\n  "json_file": "/Users/ceb/git/docling-mcp/_cache/6f1933d17fb2cce1d2a578e40315f711.json"\n}',
│   │   type='text'
│   )
]


---------- 📍 Step 13: InferenceStep ----------
🤖 Model Response:
{
  "thought": "I want to save the document",
  "action": {
    "tool_name": "save_docling_document",
    "tool_params": [
      {"name": "document_key", "value": "6f1933d17fb2cce1d2a578e40315f711"}
    ]
  },
  "answer": "/Users/ceb/git/docling-mcp/_cache/6f1933d17fb2cce1d2a578e40315f711.md"
}

========== Query processing completed ========== 

